<center>
    <img src="img/CMS_Jets.png" width="40%" />
    <br />
    <h1>Implementing a Charm Tagger with Scikit-Learn</h1>
    <br /><br />
    Seth Moortgat, December 14, 2015
    <br /><br />
    Machine Learning Seminar @ IIHE
</center>

<center>
    <h1> Overview: What is a charm tagger?
    <img src="img/CSV.png" width="35%" />
</center>
* Charm tagging in CMS: Exploit the lifetime of D mesons 
→ travels some distance in the tracker before it decays = secondary vertex (SV) with displaced tracks
* Combine information from Secondary Vertices, displaced tracks and soft leptons inside the jet to identify charm-quark jets and discriminate them from bottom- or light-flavour jets.

<center>
    <h1> Use Multivariate Analysis (MVA) techniques
    <img src="img/MVA.png" width="80%" />
</center>

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["figure.max_open_warning"] = -1

# Print options
import numpy as np
np.set_printoptions(precision=3)

In [ ]:
variables = [
    # Displaced Tracks
  "trackSip2dSig_0",
  "trackSip3dSig_0",
  "trackPtRel_0",
  "trackPPar_0",
  "trackEtaRel_0",
  "trackDeltaR_0",
  "trackPtRatio_0",
  "trackPParRatio_0",
  "trackJetDist_0",
  "trackDecayLenVal_0",
  "trackSip2dSigAboveCharm_0",
  "trackSip3dSigAboveCharm_0",
  "trackSumJetEtRatio",
  "trackSumJetDeltaR",
    # Secondary Vertex
  "vertexMass_0",
  "vertexEnergyRatio_0",
  "flightDistance2dSig_0",
  "flightDistance3dSig_0",
  "vertexJetDeltaR_0",
  "massVertexEnergyFraction_0",
  "vertexBoostOverSqrtJetPt_0",
  "jetNSecondaryVertices",
  "jetNTracks",
  "vertexNTracks_0",
    # Soft Leptons
  "leptonPtRel_0",
  "leptonSip3d_0",
  "leptonDeltaR_0",
  "leptonRatioRel_0",
  "leptonEtaRel_0",
  "leptonRatio_0",
  ]

## Discriminate charm-jets from light jets

In [ ]:
signal_files = [ # C = charm
    "./data/flat_trees/root_files/skimmed_20k_eachptetabin_CombinedSVNoVertex_C.root",
    "./data/flat_trees/root_files/skimmed_20k_eachptetabin_CombinedSVPseudoVertex_C.root",
    "./data/flat_trees/root_files/skimmed_20k_eachptetabin_CombinedSVRecoVertex_C.root"
    ]
bckgr_files = [  # DUSG = light
    "./data/flat_trees/root_files/skimmed_20k_eachptetabin_CombinedSVNoVertex_DUSG.root",
    "./data/flat_trees/root_files/skimmed_20k_eachptetabin_CombinedSVPseudoVertex_DUSG.root",
    "./data/flat_trees/root_files/skimmed_20k_eachptetabin_CombinedSVRecoVertex_DUSG.root"
    ]

## Importing ROOT trees using root_numpy
Need to have a working version of ROOT and root_numpy installed

This allows you you to read in root files and convert them to numpy-type arrays needed for the scikit-learn training
* [recarray](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.recarray.html)
* [ndarray](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html)

In [ ]:
print 'Merging and converting the samples'
nfiles_per_sample = None
skip_n_events = None

import root_numpy as rootnp

#root_numpy.root2array(filenames, treename=None, branches=None, selection=None, start=None, stop=None, step=None, include_weight=False, weight_name='weight', cache_size=-1)
signal_merged = np.ndarray((0,len(variables)),float)
bckgr_merged = np.ndarray((0,len(variables)),float)
for f_sig in signal_files:
	signal = rootnp.root2array(f_sig,'tree',variables,None,0,nfiles_per_sample,skip_n_events,False,'weight')
	signal = rootnp.rec2array(signal)
	signal_merged = np.concatenate((signal_merged,signal),0)
for f_bck in bckgr_files:
	bckgr = rootnp.root2array(f_bck,'tree',variables,None,0,nfiles_per_sample,skip_n_events,False,'weight')
	bckgr = rootnp.rec2array(bckgr)
	bckgr_merged = np.concatenate((bckgr_merged,bckgr),0)

In [ ]:
# if you don't have root_numpy --> load the python arrays (10 times less events)
from sklearn.externals import joblib
signal_files = [ # C = charm
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVNoVertex_C.root_list.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVPseudoVertex_C.root_list.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVRecoVertex_C.root_list.pkl"
    ]
bckgr_files = [  # DUSG = light
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVNoVertex_DUSG.root_list.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVPseudoVertex_DUSG.root_list.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVRecoVertex_DUSG.root_list.pkl"
    ]

print 'Merging and converting the samples'
nfiles_per_sample = None
skip_n_events = None
signal_merged = np.ndarray((0,len(variables)),float)
bckgr_merged = np.ndarray((0,len(variables)),float)
for f_sig in signal_files:
	signal = joblib.load(f_sig)
	signal_merged = np.concatenate((signal_merged,signal),0)
for f_bck in bckgr_files:
	bckgr = joblib.load(f_bck)
	bckgr_merged = np.concatenate((bckgr_merged,bckgr),0)

In [ ]:
X = np.concatenate((signal_merged, bckgr_merged))
y = np.concatenate((np.ones(signal_merged.shape[0]),np.zeros(bckgr_merged.shape[0])))
#print X[:10]
#print 'signal:',y[1], 'bckgr:', y[-1]

## Apply weights from a branch called 'weight'
* here we will simply read out a branch called 'weight' that contains the event weights
* root_numpy can also read weights from a tree if the TTree::SetWeight() member function was used

In [ ]:
print 'Getting event weights from the trees'
# Get the weights
weights = np.ones(0)
for f_sig in signal_files:
	weights_sig = rootnp.root2array(f_sig,'tree','weight',None,0,nfiles_per_sample,skip_n_events,False,'weight')
	weights = np.concatenate((weights,weights_sig),0)
for f_bck in bckgr_files:	
	weights_bckgr = rootnp.root2array(f_bck,'tree','weight',None,0,nfiles_per_sample,skip_n_events,False,'weight')
	weights = np.concatenate((weights,weights_bckgr),0)
#np.set_printoptions(precision=6)
#print weights[:10]

In [ ]:
# if you don't have root_numpy --> load the python arrays for the weights
signal_files_weights = [ # C = charm
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVNoVertex_C.root_list_weights.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVPseudoVertex_C.root_list_weights.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVRecoVertex_C.root_list_weights.pkl"
    ]
bckgr_files_weights = [  # DUSG = light
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVNoVertex_DUSG.root_list_weights.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVPseudoVertex_DUSG.root_list_weights.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVRecoVertex_DUSG.root_list_weights.pkl"
    ]

print 'Merging and converting the samples'
nfiles_per_sample = None
skip_n_events = None
weights = np.ones(0)
for f_sig in signal_files_weights:
	weights_sig = joblib.load(f_sig)
	weights = np.concatenate((weights,weights_sig),0)
for f_bck in bckgr_files_weights:
	weights_bck = joblib.load(f_bck)
	weights = np.concatenate((weights,weights_bck),0)

## Top feature (variable) selection
* You can let sklearn determine the [top-n features](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) out of a larger set of variables

* The classifier (MVA-method) needs to have a ranking defined

* It will recursively train on the variables and drop the lowest ranked variable each time

* other feature selection methods can be found [here](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection).

In [ ]:
import time
from sklearn.ensemble import RandomForestClassifier 
#BDT with 10 trees, minimum 10 events to split a node, running 5 jobs at the same time
clf = RandomForestClassifier(n_estimators=10,min_samples_split = 10,n_jobs = 1,verbose = 0)
# feature_selection is replaced by model_selection: only available in the sklearn 0.18dev versionable ranking
from sklearn.feature_selection import RFE
# select the top 20 features 
feature_selector = RFE(clf, n_features_to_select=20, verbose=1)

In [ ]:
start = time.time()
feature_selector.fit(X, y)
end = time.time()
print 'training completed --> Elapsed time: ' , (end-start)/60 ,  'minutes'

In [ ]:
# print the final set of parameters (get_support() return an arays of true/false for your variables)
print 'variables = ['
for idx,ft in enumerate(feature_selector.get_support()):
	if ft:
		print '\t\''+variables[idx]+'\','
print ']'		

In [ ]:
variables = [
	'trackSip2dSig_0',
	'trackSip3dSig_0',
	'trackPtRel_0',
	'trackPPar_0',
	'trackDeltaR_0',
	'trackPtRatio_0',
	'trackPParRatio_0',
	'trackJetDist_0',
	'trackDecayLenVal_0',
	'trackSip2dSigAboveCharm_0',
	'trackSip3dSigAboveCharm_0',
	'trackSumJetEtRatio',
	'trackSumJetDeltaR',
	'vertexEnergyRatio_0',
	'flightDistance2dSig_0',
	'vertexBoostOverSqrtJetPt_0',
	'jetNTracks',
	'leptonSip3d_0',
	'leptonRatioRel_0',
	'leptonRatio_0',
]


## re-read your trees with these 20 variables

In [ ]:
print 'Merging and converting the samples'

import root_numpy as rootnp

#root_numpy.root2array(filenames, treename=None, branches=None, selection=None, start=None, stop=None, step=None, include_weight=False, weight_name='weight', cache_size=-1)
signal_merged = np.ndarray((0,len(variables)),float)
bckgr_merged = np.ndarray((0,len(variables)),float)
for f_sig in signal_files:
	signal = rootnp.root2array(f_sig,'tree',variables,None,0,nfiles_per_sample,skip_n_events,False,'weight')
	signal = rootnp.rec2array(signal)
	signal_merged = np.concatenate((signal_merged,signal),0)
for f_bck in bckgr_files:
	bckgr = rootnp.root2array(f_bck,'tree',variables,None,0,nfiles_per_sample,skip_n_events,False,'weight')
	bckgr = rootnp.rec2array(bckgr)
	bckgr_merged = np.concatenate((bckgr_merged,bckgr),0)
    
X = np.concatenate((signal_merged, bckgr_merged))
y = np.concatenate((np.ones(signal_merged.shape[0]),np.zeros(bckgr_merged.shape[0])))

In [ ]:
# if you don't have root_numpy --> load the python arrays
from sklearn.externals import joblib
signal_files = [ # C = charm
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVNoVertex_C.root_list_top20.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVPseudoVertex_C.root_list_top20.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVRecoVertex_C.root_list_top20.pkl"
    ]
bckgr_files = [  # DUSG = light
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVNoVertex_DUSG.root_list_top20.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVPseudoVertex_DUSG.root_list_top20.pkl",
    "./data/flat_trees/numpy_files/skimmed_20k_eachptetabin_CombinedSVRecoVertex_DUSG.root_list_top20.pkl"
    ]

print 'Merging and converting the samples'
nfiles_per_sample = None
skip_n_events = None
signal_merged = np.ndarray((0,len(variables)),float)
bckgr_merged = np.ndarray((0,len(variables)),float)
for f_sig in signal_files:
	signal = joblib.load(f_sig)
	signal_merged = np.concatenate((signal_merged,signal),0)
for f_bck in bckgr_files:
	bckgr = joblib.load(f_bck)
	bckgr_merged = np.concatenate((bckgr_merged,bckgr),0)
    
X = np.concatenate((signal_merged, bckgr_merged))
y = np.concatenate((np.ones(signal_merged.shape[0]),np.zeros(bckgr_merged.shape[0])))

## Automatic selection of the most optimal MVA setting
* Scikit-Learn can [scan over a range of settings for your MVA](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and choose the one with the smallest 'error'
* the error is based on a [scorer function](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html#sklearn.metrics.make_scorer) (accuracy_score, mean_squared_error, adjusted_rand_index, average_precision)
* Only available in sklearn 0.18dev version

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':list([5,10,20])}#,'min_samples_split':list([50,100])}
rfc = RandomForestClassifier()
start = time.time()
clf = GridSearchCV(rfc, parameters, n_jobs=1, verbose=3)
clf.fit(X,y,weights)
end = time.time()
print("Best parameters = %s" % (clf.best_params_))

# Training on optimized RFC with top-20 variables

In [ ]:
clf = RandomForestClassifier(n_estimators=20,n_jobs = 1, verbose = 3)
clf.fit(X, y,weights)

# Run validation on a different set of samples

In [ ]:
val_signal_files = [
    "./data/flat_trees/root_files/CombinedSVNoVertex_C.root",
    "./data/flat_trees/root_files/CombinedSVPseudoVertex_C.root",
    "./data/flat_trees/root_files/CombinedSVRecoVertex_C.root"
    ] 
val_bckgr_files = [
    "./data/flat_trees/root_files/CombinedSVNoVertex_DUSG.root",
    "./data/flat_trees/root_files/CombinedSVPseudoVertex_DUSG.root",
    "./data/flat_trees/root_files/CombinedSVRecoVertex_DUSG.root"
    ]


print 'Preparing validation'

#root_numpy.root2array(filenames, treename=None, branches=None, selection=None, start=None, stop=None, step=None, include_weight=False, weight_name='weight', cache_size=-1)
val_signal_merged = np.ndarray((0,len(variables)),float)
val_bckgr_merged = np.ndarray((0,len(variables)),float)
for f_sig in val_signal_files:
	val_signal = rootnp.root2array(f_sig,'tree',variables,None,0,nfiles_per_sample,skip_n_events,False,'weight')
	val_signal = rootnp.rec2array(val_signal)
	val_signal_merged = np.concatenate((val_signal_merged,val_signal),0)
for f_bck in val_bckgr_files:	
	val_bckgr = rootnp.root2array(f_bck,'tree',variables,None,0,nfiles_per_sample,skip_n_events,False,'weight')
	val_bckgr = rootnp.rec2array(val_bckgr)
	val_bckgr_merged = np.concatenate((val_bckgr_merged,val_bckgr),0)

X_val = np.concatenate((val_signal_merged, val_bckgr_merged))
y_val = np.concatenate((np.ones(val_signal_merged.shape[0]),np.zeros(val_bckgr_merged.shape[0])))

In [ ]:
# # if you don't have root_numpy --> load the python arrays
val_signal_files = [
    "./data/flat_trees/numpy_files/CombinedSVNoVertex_C.root_list_top20.pkl",
    "./data/flat_trees/numpy_files/CombinedSVPseudoVertex_C.root_list_top20.pkl",
    "./data/flat_trees/numpy_files/CombinedSVRecoVertex_C.root_list_top20.pkl"
    ] 
val_bckgr_files = [
    "./data/flat_trees/numpy_files/CombinedSVNoVertex_DUSG.root_list_top20.pkl",
    "./data/flat_trees/numpy_files/CombinedSVPseudoVertex_DUSG.root_list_top20.pkl",
    "./data/flat_trees/numpy_files/CombinedSVRecoVertex_DUSG.root_list_top20.pkl"
    ]


print 'Preparing validation'

#root_numpy.root2array(filenames, treename=None, branches=None, selection=None, start=None, stop=None, step=None, include_weight=False, weight_name='weight', cache_size=-1)
val_signal_merged = np.ndarray((0,len(variables)),float)
val_bckgr_merged = np.ndarray((0,len(variables)),float)
for f_sig in val_signal_files:
	val_signal = joblib.load(f_sig)
	val_signal_merged = np.concatenate((val_signal_merged,val_signal),0)
for f_bck in val_bckgr_files:	
	val_bckgr = joblib.load(f_bck)
	val_bckgr_merged = np.concatenate((val_bckgr_merged,val_bckgr),0)

X_val = np.concatenate((val_signal_merged, val_bckgr_merged))
y_val = np.concatenate((np.ones(val_signal_merged.shape[0]),np.zeros(val_bckgr_merged.shape[0])))

# Drawing the discriminator distributions from the validation sample
you can use predict(X_val) to get the most probable flavour of the jet (it will output True (c-jet) or False (light-jet))

You can use predict_proba(X_val) to get the probability of the jet being Charm or Light
* This is the discriminator of the classifier
* it will output an array or which each element is a couple: [P(false),P(true)] = [P(light),P(charm)]

In [ ]:
from tutorial import plot_histogram
plot_histogram(clf, X_val, y_val)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_val, clf.predict_proba(X_val)[:, 1])
plt.semilogy([0.001,0.01,0.1,0.2,0.3,0.5,0.8,1], [0.001,0.01,0.1,0.2,0.3,0.5,0.8,1],label='diagonal')
plt.semilogy(tpr, fpr,label='tutorial c-tagger')
plt.semilogy([0,0.1,0.2,0.3,0.4,0.5,0.6,0.8,1], [0.00001,0.002,0.01,0.04,0.1,0.2,0.3,0.6,1],label='Current c-tagger')
plt.ylabel("Light Efficiency")
plt.xlabel("Charm Efficiency")
plt.legend(loc='best')
plt.grid(True)
plt.show()